In [0]:
#There is no need to use any passwords here because the MetaStore is already configured to use the Access Connector.

from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType


races_df = spark.read
    .option("header", True)
    .csv("abfss://bronze@mjuanworkshopetl.dfs.core.windows.net/races.csv")

In [0]:
races_df.display()

In [0]:
from pyspark.sql.functions import to_timestamp, concat, lit, col


races_df = races_df.withColumn('race_timestamp', to_timestamp(concat(col('date'), 
                                                  lit(' '), 
                                                  col('time')), 
                                           'yyyy-MM-dd HH:mm:ss'))

races_df.display()

In [0]:
from pyspark.sql.functions import col

races_df_renamed = races_df.select(col('raceId').alias('race_id').cast('integer')
                                   , col('year').alias('race_year').cast('integer'), 
                                   col('round').cast('integer'), 
                                   col('circuitId').alias('circuit_id').cast('integer'), 
                                   col('name'), 
                                   col('race_timestamp'))

races_df_renamed.display()

In [0]:
from pyspark.sql.functions import current_timestamp

race_df_final = races_df_renamed.withColumn('ingestion_date', current_timestamp()) 

race_df_final.display()

In [0]:
race_df_final.printSchema()

In [0]:
#IMPORTANT. OVERWRITE SCHEMA IF YOU CHANGED YOUR SCHEMA AFTER CREATING THE TABLE, IF NOT 
#YOU WILL HAVE MISMATCHED DATA AND THE PIPELINE CAN FAIL

race_df_final.write.mode('overwrite').option("overwriteSchema", "true").saveAsTable('formula1_project.silver.races_transformed')

In [0]:
%sql

SELECT *
FROM formula1_project.silver.races_transformed
WHERE circuit_id < 20